In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd
import numpy as np
from pathlib import Path

# Functions

In [2]:
def weighted_mean(x, metric: str):
    if len(x[metric].dropna()) == 0:
        res = 0
    else:
        res = (x[metric] * x["sample_weight"]).sum()/x["sample_weight"].sum()
    col_name = f"{metric} wt_mean"
    return pd.Series({col_name: res})

def weighted_percentile(x, metric: str, percents: int):
    col_name = f"{metric} wt_p{percents}"
    res = _weighted_percentile(x[metric], percents, weights=x["sample_weight"])
    return pd.Series({col_name: res})

def _weighted_percentile(data, percents: int, weights=None):
    """
    perc : percentile in 0 - 1
    """
    # dropna
    idx = ~np.isnan(data) & ~np.isnan(weights)
    data = np.array(data)[idx]
    if len(data) == 0:
        return np.nan
    if weights is None:
        return np.percentile(data, percents)
    weights = np.array(weights)[idx]
    ix = np.argsort(data)
    data = data[ix] # sort data
    weights = weights[ix] # sort weights
    cdf = (np.cumsum(weights) - 0.5 * weights) / np.sum(weights) # 'like' a CDF function
    return np.interp(percents, cdf, data)

def get_summary(df, metric, lower_percentile=0.25, upper_percentile=0.75):
    dfgb = df.groupby(["upgrade_name"])
    dfs = pd.concat([
        dfgb["sample_weight"].count().rename("n_samples"),
        dfgb["sample_weight"].sum().rename("n_represented"),
        dfgb.apply(lambda x: weighted_mean(x, metric)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=lower_percentile)),
        dfgb.apply(lambda x: weighted_percentile(x, metric, percents=upper_percentile)),
    ], axis=1)
    dfs = dfs.sort_values(by=f"{metric} wt_mean", ascending=False)

    return dfs

## Whole Community Upgrade Cost Per Dwelling Unit

In [3]:
def community_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']/results['sample_weight']
    results = results.sort_values('upgrade_cost_usd')
    results['upgrade_cost_usd'] = results['upgrade_cost_usd'].round(1) 
    
    print('upgrade_cost_usd per dwelling unit')
    display(results['upgrade_cost_usd'])
    return results['upgrade_cost_usd']

## Whole Community Total Annual Savings

In [4]:
# flavor 1 - with electricity and non-electricity split
def figure_total_savings_1(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.electricity_kwh',
                      'saving_energy.natural_gas_therm',
                                'saving_energy.fuel_oil_mmbtu',
                                'saving_energy.propane_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=5, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=5)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission<br>Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Electricity<br>Savings<br>kwh", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Gas, Propane,<br>Fuel Oil Savings<br>mmbtu", xaxis5_visible=True, yaxis5_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=900,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_1.svg')    

# flavor 2 - site, electricity, and non-electricity
def figure_total_savings_2(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_energy.electricity_kwh',
        'saving_energy.natural_gas_therm',
        'saving_energy.fuel_oil_mmbtu',
        'saving_energy.propane_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_therm']*0.1 + results['saving_energy.fuel_oil_mmbtu']+ results['saving_energy.propane_mmbtu']
    results['saving_energy.electricity_kwh'] = results['saving_energy.electricity_kwh']*results['sample_weight']
    results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'] = results['saving_energy.natural_gas_fuel_oil_propane_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.electricity_kwh': 'sum',
                     'saving_energy.natural_gas_fuel_oil_propane_mmbtu': 'sum', 
                      'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=6, horizontal_spacing=0.12, column_widths=[0.001, 1, 1, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)

    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.electricity_kwh'],
        y=results['upgrade_name'],
        text=results['saving_energy.electricity_kwh'],
        orientation='h'),row=1, col=5)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.natural_gas_fuel_oil_propane_mmbtu'],
        orientation='h'),row=1, col=6)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=5)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=6)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill<br>Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Site Emission<br>Saving<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Site Energy<br>Saving<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                      xaxis5_title="Electricity<br>Saving<br>kwh", xaxis5_visible=True, yaxis5_visible=False,
                      xaxis6_title="Gas, Propane,<br>Fuel Oil Saving<br>mmbtu", xaxis6_visible=True, yaxis6_visible=False,
                     title='Community Total Annual Savings')
    fig.update_layout(
    autosize=False,
    width=1100,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_2.svg')
    
    return results[['saving_cbill.total_usd','saving_emission.total_kgCO2e','saving_energy.total_mmbtu','saving_energy.electricity_kwh','saving_energy.natural_gas_fuel_oil_propane_mmbtu']]

# flavor 3 - site energy only
def figure_total_savings_3(commnity_results):
    results = commnity_results[[
        'upgrade_name',
        'saving_energy.total_mmbtu',
        'saving_cbill.total_usd',
        'saving_emission.total_kgCO2e',
        'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results = results.sort_values('saving_cbill.total_usd')
    
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Total Annual Savings for the Whole Community')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'total_annual_savings_3.svg')

## Whole Community Annual Savings Per Dewlling Unit

In [5]:
def figure_mean_savings(commnity_results):
    results = commnity_results[['upgrade_name',
                       'saving_energy.total_mmbtu',
                       'saving_cbill.total_usd',
                       'saving_emission.total_kgCO2e',
                                'sample_weight']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']*results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']*results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']*results['sample_weight']
    
    agg_functions = {'upgrade_name': 'first',
                     'saving_energy.total_mmbtu': 'sum', 
                     'saving_cbill.total_usd': 'sum',
                     'saving_emission.total_kgCO2e': 'sum',
                    'sample_weight': 'sum'}
    results = results.groupby(results['upgrade_name']).aggregate(agg_functions)
    results['saving_energy.total_mmbtu'] = results['saving_energy.total_mmbtu']/results['sample_weight']
    results['saving_cbill.total_usd'] = results['saving_cbill.total_usd']/results['sample_weight']
    results['saving_emission.total_kgCO2e'] = results['saving_emission.total_kgCO2e']/results['sample_weight']
    results = results.sort_values('saving_cbill.total_usd')
    
    print('sample_weight')
    display(results['sample_weight'])
  
    fig = make_subplots(rows=1, cols=4, horizontal_spacing=0.1, column_widths=[0.02, 1, 1, 1])
    
    fig.add_trace(go.Bar(
        x=[0] * 176,
        y=results['upgrade_name'],
        text=None,
        orientation='h'), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=results['saving_cbill.total_usd'],
        y=results['upgrade_name'],
        text=results['saving_cbill.total_usd'],
        orientation='h'), row=1, col=2)
    
    fig.add_trace(go.Bar(
        x=results['saving_emission.total_kgCO2e'],
        y=results['upgrade_name'],
        text=results['saving_emission.total_kgCO2e'],
        orientation='h'),row=1, col=3)
    
    fig.add_trace(go.Bar(
        x=results['saving_energy.total_mmbtu'],
        y=results['upgrade_name'],
        text=results['saving_energy.total_mmbtu'],
        orientation='h'),row=1, col=4)

    fig.update_traces(textfont_size=10, textangle=0, textposition="outside", cliponaxis=False)
    fig.update_traces(marker_color='rgb(0, 121, 61)', marker_line_color='rgb(0, 121, 61)')
    fig.update_traces(marker_color='rgb(255, 255, 255)', marker_line_color='rgb(255, 255, 255)',row=1,col=1)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=2)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=3)
    fig.update_traces(texttemplate='%{text:.3s}', textposition='outside',row=1,col=4)
    fig.update_layout(xaxis_title="", xaxis_visible=False, yaxis_visible=True,
                      xaxis2_title="Bill Savings<br>usd", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Emission Savings<br>kgCO2e", xaxis3_visible=True, yaxis3_visible=False,
                      xaxis4_title="Energy Savings<br>mmbtu", xaxis4_visible=True, yaxis4_visible=False,
                     title='Annual Savings Per Dwelling Unit')
    fig.update_layout(
    autosize=False,
    width=850,
    height=500,
    font=dict(size=10),
    plot_bgcolor='rgba(0, 0, 0, 0)',)
    fig.update_xaxes(tickangle=45)
    fig.update_layout(showlegend=False)
    
    fig.show()
    fig.write_image(fig_dir / 'annual_saving_per_dwelling_unit.svg')
    
    return results[['saving_cbill.total_usd','saving_emission.total_kgCO2e','saving_energy.total_mmbtu']]

## Segment Upgrade Cost Per Dwelling Unit

In [6]:
def segment_mean_upgrade_cost(commnity_results):
    results = commnity_results[['upgrade_name',
                       'upgrade_cost_usd',
                                'sample_weight',
                                'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     'segment': 'first',
                     'upgrade_cost_usd': 'sum',
                    'sample_weight': 'sum'}
    
    segment1 = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2 = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3 = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4 = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd']/segment1['sample_weight']
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd']/segment2['sample_weight']
    segment3['upgrade_cost_usd'] = segment3['upgrade_cost_usd']/segment3['sample_weight']
    segment4['upgrade_cost_usd'] = segment4['upgrade_cost_usd']/segment4['sample_weight']
    
    segment1 = segment1.sort_values('upgrade_cost_usd')
    segment2 = segment2.sort_values('upgrade_cost_usd')
    segment3 = segment3.sort_values('upgrade_cost_usd')
    segment4 = segment4.sort_values('upgrade_cost_usd')
    
    segment1['upgrade_cost_usd'] = segment1['upgrade_cost_usd'].round(1)
    segment2['upgrade_cost_usd'] = segment2['upgrade_cost_usd'].round(1)
    segment3['upgrade_cost_usd'] = segment3['upgrade_cost_usd'].round(1)
    segment4['upgrade_cost_usd'] = segment4['upgrade_cost_usd'].round(1)
    
    print('segment1: upgrade cost (usd) per dwelling unit')
    display(segment1['upgrade_cost_usd'])
    print('segment2: upgrade cost (usd) per dwelling unit')
    display(segment2['upgrade_cost_usd'])
    print('segment3: upgrade cost (usd) per dwelling unit')
    display(segment3['upgrade_cost_usd'])
    print('segment4: upgrade cost (usd) per dwelling unit')
    display(segment4['upgrade_cost_usd'])
    return (segment1['upgrade_cost_usd'], segment2['upgrade_cost_usd'], segment3['upgrade_cost_usd'], segment4['upgrade_cost_usd'])

## Segment Savings Per Dollar Invested

In [7]:
def segment_savings_per_dollar(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'upgrade_cost_usd', 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    results['upgrade_cost_usd'] = results['upgrade_cost_usd']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'upgrade_cost_usd': 'sum',
                     'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['upgrade_cost_usd']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['upgrade_cost_usd']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['upgrade_cost_usd']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['upgrade_cost_usd']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(5)
    segment2_agg[indicator] = segment2_agg[indicator].round(5)
    segment3_agg[indicator] = segment3_agg[indicator].round(5)
    segment4_agg[indicator] = segment4_agg[indicator].round(5)
    
    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    print('segment3')
    display(segment3_agg[indicator])
    print('segment4')
    display(segment4_agg[indicator]) 
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator],segment4_agg[indicator])

## Segment Annual Total Savings

In [8]:
def segment_annual_total_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)
    segment3_agg[indicator] = segment3_agg[indicator].round(1)
    segment4_agg[indicator] = segment4_agg[indicator].round(1)

    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    print('segment3')
    display(segment3_agg[indicator])
    print('segment4')
    display(segment4_agg[indicator])
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator],segment4_agg[indicator])

## Segment Annual Savings Per Dwelling Unit

In [9]:
def segment_mean_savings(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'sample_weight', 'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results[indicator] = results[indicator]*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['sample_weight']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['sample_weight']
    
    segment1_agg = segment1_agg.sort_values([indicator], ascending=False)
    segment2_agg = segment2_agg.sort_values([indicator], ascending=False)
    segment3_agg = segment3_agg.sort_values([indicator], ascending=False)
    segment4_agg = segment4_agg.sort_values([indicator], ascending=False)
    
    segment1_agg[indicator] = segment1_agg[indicator].round(1)
    segment2_agg[indicator] = segment2_agg[indicator].round(1)
    segment3_agg[indicator] = segment3_agg[indicator].round(1)
    segment4_agg[indicator] = segment4_agg[indicator].round(1)
    
    print('segment1')
    display(segment1_agg[indicator])
    print('segment2')
    display(segment2_agg[indicator])
    print('segment3')
    display(segment3_agg[indicator])
    print('segment4')
    display(segment4_agg[indicator])
    
    return (segment1_agg[indicator],segment2_agg[indicator],segment3_agg[indicator],segment4_agg[indicator])

## Segment Percent Stock with Net Positive Savings

In [10]:
def figure_positive_negative_savings(segment, commnity_results):
    results = commnity_results[['upgrade_name',
                                  'saving_energy.total_mmbtu',
                                  'saving_cbill.total_usd',
                                  'saving_emission.total_kgCO2e',
                                'sample_weight',
                               'segment']]
    results = results.loc[results['upgrade_name'] != 'baseline']
    results = results.loc[results['segment'] == segment]
    results = results.groupby(results['upgrade_name'])
  
    def pos(col): 
      return col[col > 0].count()
  
    def neg(col): 
      return col[col < 0].count() 

    percentage = pd.DataFrame()

    bill = results['saving_cbill.total_usd'].agg([('negative_values', neg),('positive_values', pos)])
    emission = results['saving_emission.total_kgCO2e'].agg([('negative_values', neg),('positive_values', pos)])
    energy = results['saving_energy.total_mmbtu'].agg([('negative_values', neg),('positive_values', pos)])
    
    percentage ['cbill_n'] = bill['negative_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['cbill_p'] = bill['positive_values']/(bill['negative_values']+bill['positive_values'])
    percentage ['emission_n'] = emission['negative_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['emission_p'] = emission['positive_values']/(emission['negative_values']+emission['positive_values'])
    percentage ['energy_n'] = energy['negative_values']/(energy['negative_values']+energy['positive_values'])
    percentage ['energy_p'] = energy['positive_values']/(energy['negative_values']+energy['positive_values'])
    
    
    percentage = percentage.sort_values(('cbill_p'))
    upgrade_name = percentage.index
    
    fig = make_subplots(rows=1, cols=3)
    marker1= dict(color='red')
    marker2= dict(color='green')
    
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_n'], text=percentage ['cbill_n'],
                orientation='h',name = "Negative",row=1, col=1, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['cbill_p'],text=percentage ['cbill_p'],
                orientation='h', name = "Positive", row=1, col=1, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['emission_n'], text=percentage ['emission_n'],
                orientation='h',showlegend=False, row=1, col=2, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['emission_p'],text=percentage ['emission_p'],
                orientation='h', showlegend=False, row=1, col=2, marker=marker2)
    
    fig.add_bar(y=upgrade_name, x=percentage ['energy_n'],text=percentage ['energy_n'],
                orientation='h',showlegend=False, row=1, col=3, marker=marker1)
    fig.add_bar(y=upgrade_name, x=percentage ['energy_p'],text=percentage ['energy_p'],
                orientation='h', showlegend=False, row=1, col=3, marker=marker2)
   
  
    fig.update_layout(barmode="relative")
    fig.update_traces(texttemplate='%{text:.0%}', textposition='inside')
    
    fig.update_layout(xaxis_title="Bill", xaxis_visible=True, yaxis_visible=True,
                      xaxis2_title="Emission", xaxis2_visible=True, yaxis2_visible=False,
                      xaxis3_title="Total Energy", xaxis3_visible=True, yaxis3_visible=False,
                      font=dict(size=10))
    
    fig.update_layout(
        title='Percentage of Household in Positive and Negative Savings')
    
    fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    font=dict(size=10))
    fig.layout.xaxis1.tickformat = ',.0%'
    fig.layout.xaxis2.tickformat = ',.0%'
    fig.layout.xaxis3.tickformat = ',.0%'
    
    fig.show()
    fig.write_image(fig_dir / f'percentage_positive_saving_{segment}.svg')
    return (percentage ['cbill_p'],percentage ['emission_p'],percentage ['energy_p'])

## Segment Average Energy Burden (%) Before and after Upgrade.

In [11]:
def segment_mean_energy_burden(indicator, commnity_results):
    
    results = commnity_results[['upgrade_name', indicator, 'baseline_energy_burden_2023_cbills.%', 'sample_weight', 'segment']]
    results.replace([np.inf, -np.inf], np.nan, inplace=True)
    results.dropna(subset=['baseline_energy_burden_2023_cbills.%'], how="all", inplace=True)
    results[indicator] = results[indicator]*results['sample_weight']
    results['baseline_energy_burden_2023_cbills.%'] = results['baseline_energy_burden_2023_cbills.%']*results['sample_weight']
    
    segment1 = results.loc[results['segment'] == 'segment1']
    segment2 = results.loc[results['segment'] == 'segment2']
    segment3 = results.loc[results['segment'] == 'segment3']
    segment4 = results.loc[results['segment'] == 'segment4']
    
    agg_functions = {'upgrade_name': 'first',
                     indicator: 'sum',
                     'baseline_energy_burden_2023_cbills.%': 'sum',
                    'sample_weight': 'sum'}
    
    segment1_agg = segment1.groupby(segment1['upgrade_name']).aggregate(agg_functions)
    segment2_agg = segment2.groupby(segment2['upgrade_name']).aggregate(agg_functions)
    segment3_agg = segment3.groupby(segment3['upgrade_name']).aggregate(agg_functions)
    segment4_agg = segment4.groupby(segment4['upgrade_name']).aggregate(agg_functions)
    
    segment1_agg['baseline_energy_burden_2023_cbills.%'] = segment1_agg['baseline_energy_burden_2023_cbills.%']/segment1_agg['sample_weight']
    segment2_agg['baseline_energy_burden_2023_cbills.%'] = segment2_agg['baseline_energy_burden_2023_cbills.%']/segment2_agg['sample_weight']
    segment3_agg['baseline_energy_burden_2023_cbills.%'] = segment3_agg['baseline_energy_burden_2023_cbills.%']/segment3_agg['sample_weight']
    segment4_agg['baseline_energy_burden_2023_cbills.%'] = segment4_agg['baseline_energy_burden_2023_cbills.%']/segment4_agg['sample_weight']
    
    segment1_agg[indicator] = segment1_agg[indicator]/segment1_agg['sample_weight']
    segment2_agg[indicator] = segment2_agg[indicator]/segment2_agg['sample_weight']
    segment3_agg[indicator] = segment3_agg[indicator]/segment3_agg['sample_weight']
    segment4_agg[indicator] = segment4_agg[indicator]/segment4_agg['sample_weight']
    
    segment1_agg['energy_burden_diff'] = segment1_agg['baseline_energy_burden_2023_cbills.%'] - segment1_agg[indicator]
    segment2_agg['energy_burden_diff'] = segment2_agg['baseline_energy_burden_2023_cbills.%'] - segment2_agg[indicator]
    segment3_agg['energy_burden_diff'] = segment3_agg['baseline_energy_burden_2023_cbills.%'] - segment3_agg[indicator]
    segment4_agg['energy_burden_diff'] = segment4_agg['baseline_energy_burden_2023_cbills.%'] - segment4_agg[indicator]
    
    segment1_agg = segment1_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment2_agg = segment2_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment3_agg = segment3_agg.sort_values(['energy_burden_diff'], ascending=False)
    segment4_agg = segment4_agg.sort_values(['energy_burden_diff'], ascending=False)
    
    segment1_agg = segment1_agg[segment1_agg.upgrade_name != 'baseline']
    segment2_agg = segment2_agg[segment2_agg.upgrade_name != 'baseline']
    segment3_agg = segment3_agg[segment3_agg.upgrade_name != 'baseline']
    segment4_agg = segment4_agg[segment4_agg.upgrade_name != 'baseline']
    
    segment1_agg = segment1_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment2_agg = segment2_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment3_agg = segment3_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    segment4_agg = segment4_agg.rename(columns={'baseline_energy_burden_2023_cbills.%': 'before', indicator: 'after'})
    
    segment1_agg['before'] = segment1_agg['before'].round(1)
    segment2_agg['before'] = segment2_agg['before'].round(1)
    segment3_agg['before'] = segment3_agg['before'].round(1)
    segment4_agg['before'] = segment4_agg['before'].round(1)
    segment1_agg['after'] = segment1_agg['after'].round(1)
    segment2_agg['after'] = segment2_agg['after'].round(1)
    segment3_agg['after'] = segment3_agg['after'].round(1)
    segment4_agg['after'] = segment4_agg['after'].round(1)
    
    print ('energy burden (%) before and after upgrade')
    print('segment1')
    display(segment1_agg[['before', 'after']])
    print('segment2')
    display(segment2_agg[['before', 'after']])
    print('segment3')
    display(segment3_agg[['before', 'after']])
    print('segment4')
    display(segment4_agg[['before', 'after']])
    
    return(segment1_agg[['before', 'after']],segment2_agg[['before', 'after']],segment3_agg[['before', 'after']],segment4_agg[['before', 'after']])

In [12]:
community = 'columbia'
fig_dir = Path(".").resolve() / "results" / community
community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results_TOU.parquet")
saving_results = pd.DataFrame()

# 1. Results for Whole Community

## 1.1 Upgrade Cost Per Dwelling Unit

In [13]:
results = community_mean_upgrade_cost(community_results)
saving_results['whole_community_upgrade_cost_mean_usd'] = results

upgrade_cost_usd per dwelling unit


upgrade_name
Electric Cooking                                                    1115.8
Electric Clothes Dryer                                              1253.0
Induction Cooking                                                   1918.7
Heat Pump Clothes Dryer                                             2521.5
Heat Pump Water Heater                                              3128.7
Basic Enclosure                                                     4373.6
Enhanced Enclosure                                                  6306.9
Mininum Efficiency Whole Home Electrification                      13639.7
Mininum Efficiency Heat Pump with Existing Heat Backup             14026.2
Mininum Efficiency Heat Pump with Electric Heat Backup             14128.6
High Efficiency Heat Pump with Electric Heat Backup                26553.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        30947.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     32253.8
High Efficie

## 1.2 Total Annual Savings

In [14]:
figure_total_savings_1(community_results)

In [15]:
results = figure_total_savings_2(community_results)
saving_results[['whole_community_saving_bill_total_usd','whole_community_saving_emission_total_kgCO2e','whole_community_saving_energy_total_mmbtu','whole_community_saving_electricity_total_kwh','whole_community_saving_natural_gas_fuel_oil_propane_total_mmbtu']] = results

In [16]:
figure_total_savings_3(community_results) 

## 1.3 Annual Savings Per Dwelling Unit

In [17]:
results = figure_mean_savings(community_results) 
saving_results[['whole_community_saving_bill_mean_usd','whole_community_saving_emission_mean_kgCO2e','whole_community_saving_energy_mean_mmbtu']] = results

sample_weight


upgrade_name
Electric Clothes Dryer                                              4096.305418
Electric Cooking                                                   18234.147833
Induction Cooking                                                  92049.895058
Heat Pump Clothes Dryer                                            78779.108592
Heat Pump Water Heater                                             92144.655665
Mininum Efficiency Whole Home Electrification                      38136.075473
Mininum Efficiency Heat Pump with Existing Heat Backup             92755.987940
Mininum Efficiency Heat Pump with Electric Heat Backup             93536.674242
Basic Enclosure                                                    91597.180135
Enhanced Enclosure                                                 92376.665224
High Efficiency Heat Pump with Electric Heat Backup                93536.674242
High Efficiency Whole Home Electrification                         93698.000000
Basic Enclosure + HPWH + Hi

In [18]:
metric = "pct_saving_energy.total_%"
print("Community summary:")
get_summary(community_results, metric, lower_percentile=0.25, upper_percentile=0.75)

Community summary:


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,1120,93698.000000,51.880599,41.956560,62.805880
Basic Enclosure + High Efficiency Whole Home Electrification,1120,93698.000000,51.100352,41.372864,62.127911
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,1120,93698.000000,50.061886,39.655341,61.691443
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,1120,93698.000000,49.281746,38.811163,61.047052
High Efficiency Whole Home Electrification,1120,93698.000000,46.847475,38.450000,55.936199
High Efficiency Heat Pump with Electric Heat Backup,1117,93536.674242,34.745254,24.652401,46.564562
Mininum Efficiency Whole Home Electrification,348,38136.075473,27.435292,5.611356,45.142413
Enhanced Enclosure,1105,92376.665224,17.494616,7.265421,25.665255
Mininum Efficiency Heat Pump with Electric Heat Backup,1117,93536.674242,17.146593,7.424104,28.581836


# 2. Results for Segments (All Income)

Segment Definition

Segment 1. Occupied, Single family detached, 1940-1979, all income 

Segment 2. Occupied, Multifamily with 5+ units, 1940-1979, all income 

Segment 3. Occupied, Single family detached, 1980+, all income 

Segment 4. Occupied, Multifamily with 5+ units, 1980+, all income 


In [19]:
income = 'all_incomes'
vacancy_status = ['Occupied']

seg1_building_type = ['Single-Family Detached']
seg1_vintage = ['1940s', '1950s', '1960s', '1970s']
seg1_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg2_building_type = ['Multi-Family with 5+ Units']
seg2_vintage = ['1940s', '1950s', '1960s', '1970s']
seg2_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg3_building_type = ['Single-Family Detached']
seg3_vintage = ['1980s', '1990s', '2000s', '2010s']
seg3_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

seg4_building_type = ['Multi-Family with 5+ Units']
seg4_vintage = ['1980s', '1990s', '2000s', '2010s']
seg4_median_income = ['0-30%','30-60%','60-80%','80-100%', '100-120%','120-150%']

community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg1_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg1_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg1_median_income),
                     'segment'] = 'segment1'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg2_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg2_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg2_median_income),
                     'segment'] = 'segment2'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg3_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg3_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg3_median_income),
                     'segment'] = 'segment3'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg4_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg4_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg4_median_income),
                     'segment'] = 'segment4'

In [20]:
metric = 'pct_saving_energy.total_%'  # TODO 'pct_saving_cbill.total_%' #'pct_saving_energy.total_%' #'saving_energy.total_mmbtu' #'saving_cbill.total_usd'

for segment in sorted(community_results["segment"].dropna().unique()):
    print(f"Summary for segment: {segment}: ")
    df_seg = community_results.loc[community_results["segment"]==segment]
    display(get_summary(df_seg, metric, lower_percentile=0.25, upper_percentile=0.75))

Summary for segment: segment1: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,133,17555.110438,62.234227,55.904250,69.167063
Basic Enclosure + High Efficiency Whole Home Electrification,133,17555.110438,61.573800,55.763750,68.842778
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,133,17555.110438,60.449792,54.531500,67.676222
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,133,17555.110438,59.788968,54.410000,67.456528
High Efficiency Whole Home Electrification,133,17555.110438,55.495694,48.301375,63.052083
High Efficiency Heat Pump with Electric Heat Backup,133,17555.110438,44.088481,36.304000,52.294398
Mininum Efficiency Whole Home Electrification,79,11433.335690,34.979913,18.980167,49.377556
Enhanced Enclosure,133,17555.110438,27.254203,20.252667,35.726270
Mininum Efficiency Heat Pump with Electric Heat Backup,133,17555.110438,27.025288,18.111818,35.915694


Summary for segment: segment2: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,88,8069.342127,43.708095,36.178942,50.462762
Basic Enclosure + High Efficiency Whole Home Electrification,88,8069.342127,42.886457,36.166509,49.877057
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,88,8069.342127,40.904972,33.606187,47.104039
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,88,8069.342127,40.083389,33.606187,45.899405
High Efficiency Whole Home Electrification,88,8069.342127,39.904118,34.601418,44.979946
High Efficiency Heat Pump with Electric Heat Backup,88,8069.342127,25.826280,18.204995,34.278741
Enhanced Enclosure,88,8069.342127,10.684951,4.328627,15.905353
Mininum Efficiency Heat Pump with Electric Heat Backup,88,8069.342127,10.402477,5.876623,16.315672
Mininum Efficiency Heat Pump with Existing Heat Backup,85,7709.536801,9.590166,5.275169,13.943813


Summary for segment: segment3: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,129,9887.441333,47.040361,40.718285,53.323627
Basic Enclosure + High Efficiency Whole Home Electrification,129,9887.441333,46.569113,40.698282,53.293237
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,129,9887.441333,44.870497,38.061937,50.456464
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,129,9887.441333,44.399342,37.455371,50.321813
High Efficiency Whole Home Electrification,129,9887.441333,43.604580,37.451223,49.578475
High Efficiency Heat Pump with Electric Heat Backup,129,9887.441333,30.509530,24.564803,37.170468
Mininum Efficiency Whole Home Electrification,34,3811.940163,20.489908,3.370437,29.565950
Mininum Efficiency Heat Pump with Electric Heat Backup,129,9887.441333,15.253867,7.068537,21.529288
Mininum Efficiency Heat Pump with Existing Heat Backup,129,9887.441333,13.405815,4.667903,20.743190


Summary for segment: segment4: 


,n_samples,n_represented,pct_saving_energy.total_% wt_mean,pct_saving_energy.total_% wt_p0.25,pct_saving_energy.total_% wt_p0.75
upgrade_name,,,,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,185,11993.040624,39.181283,34.611972,44.403706
Basic Enclosure + High Efficiency Whole Home Electrification,185,11993.040624,38.096843,34.025236,43.695263
High Efficiency Whole Home Electrification,185,11993.040624,36.446545,32.919780,41.250000
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,185,11993.040624,36.306312,30.881179,41.963407
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,185,11993.040624,35.222307,30.750000,40.887015
High Efficiency Heat Pump with Electric Heat Backup,183,11960.775472,19.275672,13.472423,25.749929
Mininum Efficiency Whole Home Electrification,33,2838.276452,12.899465,4.285723,21.672688
Heat Pump Water Heater,182,11852.917680,12.212144,7.642060,16.985154
Enhanced Enclosure,177,11267.535645,6.900381,2.368173,8.856349


## 2.1 Upgrade Cost Per Dwelling Unit 

In [21]:
results = segment_mean_upgrade_cost(community_results) 
saving_results[f'segment1_{income}_upgrade_cost_mean_usd'] = results[0]
saving_results[f'segment2_{income}_upgrade_cost_mean_usd'] = results[1]
saving_results[f'segment3_{income}_upgrade_cost_mean_usd'] = results[2]
saving_results[f'segment4_{income}_upgrade_cost_mean_usd'] = results[3]

segment1: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1115.8
Electric Clothes Dryer                                              1253.0
Induction Cooking                                                   1918.7
Heat Pump Clothes Dryer                                             2521.5
Heat Pump Water Heater                                              3133.1
Basic Enclosure                                                     7174.8
Enhanced Enclosure                                                  9912.8
Mininum Efficiency Whole Home Electrification                      16209.2
Mininum Efficiency Heat Pump with Existing Heat Backup             17741.1
Mininum Efficiency Heat Pump with Electric Heat Backup             17961.2
High Efficiency Heat Pump with Electric Heat Backup                29715.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        35012.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     36845.2
High Efficie

segment2: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1115.8
Electric Clothes Dryer                                              1253.0
Basic Enclosure                                                     1445.7
Induction Cooking                                                   1918.7
Enhanced Enclosure                                                  2342.7
Heat Pump Clothes Dryer                                             2521.5
Mininum Efficiency Whole Home Electrification                       2840.1
Heat Pump Water Heater                                              2882.5
Mininum Efficiency Heat Pump with Electric Heat Backup              9058.7
Mininum Efficiency Heat Pump with Existing Heat Backup              9068.3
High Efficiency Heat Pump with Electric Heat Backup                21567.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        24894.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     25544.0
High Efficie

segment3: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1115.8
Electric Clothes Dryer                                              1253.0
Induction Cooking                                                   1918.7
Heat Pump Clothes Dryer                                             2521.5
Heat Pump Water Heater                                              3380.9
Basic Enclosure                                                     4057.4
Enhanced Enclosure                                                  6412.4
Mininum Efficiency Whole Home Electrification                       9896.9
Mininum Efficiency Heat Pump with Existing Heat Backup             13276.2
Mininum Efficiency Heat Pump with Electric Heat Backup             13367.8
High Efficiency Heat Pump with Electric Heat Backup                27847.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        33409.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     35144.1
High Efficie

segment4: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1115.8
Basic Enclosure                                                     1206.2
Electric Clothes Dryer                                              1253.0
Induction Cooking                                                   1918.7
Enhanced Enclosure                                                  2387.0
Heat Pump Clothes Dryer                                             2521.5
Mininum Efficiency Whole Home Electrification                       2567.2
Heat Pump Water Heater                                              2890.6
Mininum Efficiency Heat Pump with Existing Heat Backup              8187.1
Mininum Efficiency Heat Pump with Electric Heat Backup              8202.4
High Efficiency Heat Pump with Electric Heat Backup                22067.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        25397.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     26371.4
High Efficie

 ## 2.2 Annual Savings Per Dollar Invested

 ### Annual Bill Savings Per Dollar Invested

In [22]:
indicator ='saving_cbill.total_usd'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_bill_per_dollar_invested_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_per_dollar_invested_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_per_dollar_invested_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_per_dollar_invested_usd'] = results[3]

segment1


upgrade_name
Basic Enclosure                                                    0.08571
Enhanced Enclosure                                                 0.07060
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.04081
Heat Pump Water Heater                                             0.04006
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.03939
Basic Enclosure + High Efficiency Whole Home Electrification       0.03731
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.03621
High Efficiency Heat Pump with Electric Heat Backup                0.03531
High Efficiency Whole Home Electrification                         0.03371
Mininum Efficiency Heat Pump with Existing Heat Backup             0.02534
Mininum Efficiency Heat Pump with Electric Heat Backup             0.02527
Mininum Efficiency Whole Home Electrification                      0.01531
Heat Pump Clothes Dryer                                            0.01478
Induction Co

segment2


upgrade_name
Basic Enclosure                                                    0.08225
Enhanced Enclosure                                                 0.06421
Heat Pump Water Heater                                             0.03909
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.02204
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.02195
Basic Enclosure + High Efficiency Whole Home Electrification       0.02000
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.01997
High Efficiency Whole Home Electrification                         0.01880
High Efficiency Heat Pump with Electric Heat Backup                0.01675
Mininum Efficiency Heat Pump with Electric Heat Backup             0.01603
Mininum Efficiency Heat Pump with Existing Heat Backup             0.01539
Heat Pump Clothes Dryer                                            0.01183
Mininum Efficiency Whole Home Electrification                      0.00907
Induction Co

segment3


upgrade_name
Basic Enclosure                                                    0.06037
Heat Pump Water Heater                                             0.04952
Enhanced Enclosure                                                 0.04795
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.03233
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.03119
Basic Enclosure + High Efficiency Whole Home Electrification       0.02980
High Efficiency Whole Home Electrification                         0.02936
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.02890
High Efficiency Heat Pump with Electric Heat Backup                0.02806
Mininum Efficiency Heat Pump with Electric Heat Backup             0.02691
Mininum Efficiency Heat Pump with Existing Heat Backup             0.02667
Heat Pump Clothes Dryer                                            0.01699
Mininum Efficiency Whole Home Electrification                      0.01180
Induction Co

segment4


upgrade_name
Basic Enclosure                                                    0.05567
Heat Pump Water Heater                                             0.05352
Enhanced Enclosure                                                 0.04575
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.01874
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.01849
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.01726
Basic Enclosure + High Efficiency Whole Home Electrification       0.01700
Mininum Efficiency Whole Home Electrification                      0.01681
High Efficiency Whole Home Electrification                         0.01655
Heat Pump Clothes Dryer                                            0.01250
High Efficiency Heat Pump with Electric Heat Backup                0.01202
Mininum Efficiency Heat Pump with Existing Heat Backup             0.01122
Mininum Efficiency Heat Pump with Electric Heat Backup             0.01102
Induction Co

 ### Annual Energy Savings Per Dollar Invested

In [23]:
indicator = 'saving_energy.total_mmbtu'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[3]

segment1


upgrade_name
Basic Enclosure                                                    0.00324
Enhanced Enclosure                                                 0.00268
Mininum Efficiency Whole Home Electrification                      0.00241
Heat Pump Water Heater                                             0.00220
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00165
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00158
Electric Cooking                                                   0.00154
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00153
Basic Enclosure + High Efficiency Whole Home Electrification       0.00151
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00145
High Efficiency Whole Home Electrification                         0.00145
High Efficiency Heat Pump with Electric Heat Backup                0.00144
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00115
Heat Pump Cl

segment2


upgrade_name
Basic Enclosure                                                    0.00224
Enhanced Enclosure                                                 0.00173
Mininum Efficiency Whole Home Electrification                      0.00138
Electric Cooking                                                   0.00138
Heat Pump Water Heater                                             0.00110
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00060
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00059
Basic Enclosure + High Efficiency Whole Home Electrification       0.00055
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00055
High Efficiency Whole Home Electrification                         0.00052
High Efficiency Heat Pump with Electric Heat Backup                0.00045
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00045
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00041
Heat Pump Cl

segment3


upgrade_name
Heat Pump Water Heater                                             0.00194
Basic Enclosure                                                    0.00172
Electric Cooking                                                   0.00168
Mininum Efficiency Whole Home Electrification                      0.00163
Enhanced Enclosure                                                 0.00136
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00098
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00094
Basic Enclosure + High Efficiency Whole Home Electrification       0.00091
High Efficiency Whole Home Electrification                         0.00090
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00088
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00086
High Efficiency Heat Pump with Electric Heat Backup                0.00081
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00076
Heat Pump Cl

segment4


upgrade_name
Mininum Efficiency Whole Home Electrification                      0.00178
Heat Pump Water Heater                                             0.00152
Basic Enclosure                                                    0.00148
Electric Cooking                                                   0.00137
Enhanced Enclosure                                                 0.00121
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00051
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00050
Electric Clothes Dryer                                             0.00049
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00047
Basic Enclosure + High Efficiency Whole Home Electrification       0.00047
High Efficiency Whole Home Electrification                         0.00046
Heat Pump Clothes Dryer                                            0.00034
High Efficiency Heat Pump with Electric Heat Backup                0.00032
Mininum Effi

 ### Annual Emission Savings Per Dollar Invested

In [24]:
indicator ='saving_emission.total_kgCO2e'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[3]

segment1


upgrade_name
Basic Enclosure                                                    0.23539
Enhanced Enclosure                                                 0.19590
Heat Pump Water Heater                                             0.15034
Mininum Efficiency Whole Home Electrification                      0.14423
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.11919
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.11476
Basic Enclosure + High Efficiency Whole Home Electrification       0.10881
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.10531
High Efficiency Heat Pump with Electric Heat Backup                0.10408
High Efficiency Whole Home Electrification                         0.10323
Mininum Efficiency Heat Pump with Electric Heat Backup             0.10302
Electric Cooking                                                   0.09892
Mininum Efficiency Heat Pump with Existing Heat Backup             0.08384
Heat Pump Cl

segment2


upgrade_name
Basic Enclosure                                                    0.18525
Enhanced Enclosure                                                 0.14520
Electric Cooking                                                   0.09383
Mininum Efficiency Whole Home Electrification                      0.08659
Heat Pump Water Heater                                             0.07801
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.04774
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.04762
Basic Enclosure + High Efficiency Whole Home Electrification       0.04380
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.04378
High Efficiency Whole Home Electrification                         0.04123
Mininum Efficiency Heat Pump with Electric Heat Backup             0.03858
High Efficiency Heat Pump with Electric Heat Backup                0.03701
Mininum Efficiency Heat Pump with Existing Heat Backup             0.03642
Heat Pump Cl

segment3


upgrade_name
Basic Enclosure                                                    0.13615
Heat Pump Water Heater                                             0.13135
Electric Cooking                                                   0.11223
Enhanced Enclosure                                                 0.10926
Mininum Efficiency Whole Home Electrification                      0.10127
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.07608
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.07342
Basic Enclosure + High Efficiency Whole Home Electrification       0.07027
High Efficiency Whole Home Electrification                         0.06988
Mininum Efficiency Heat Pump with Electric Heat Backup             0.06888
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.06815
High Efficiency Heat Pump with Electric Heat Backup                0.06467
Mininum Efficiency Heat Pump with Existing Heat Backup             0.06366
Heat Pump Cl

segment4


upgrade_name
Basic Enclosure                                                    0.12441
Mininum Efficiency Whole Home Electrification                      0.11516
Heat Pump Water Heater                                             0.10768
Enhanced Enclosure                                                 0.09473
Electric Cooking                                                   0.09385
Electric Clothes Dryer                                             0.04709
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.03919
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.03905
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.03651
Basic Enclosure + High Efficiency Whole Home Electrification       0.03630
High Efficiency Whole Home Electrification                         0.03525
High Efficiency Heat Pump with Electric Heat Backup                0.02578
Mininum Efficiency Heat Pump with Electric Heat Backup             0.02542
Mininum Effi

## 2.3 Annual Total Savings

### Annual Total Bill Savings

In [25]:
indicator ='saving_cbill.total_usd'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_total_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_total_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_total_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_total_usd'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    26188783.5
Basic Enclosure + High Efficiency Whole Home Electrification       25789865.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     25480462.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        25081543.7
High Efficiency Whole Home Electrification                         22015669.4
High Efficiency Heat Pump with Electric Heat Backup                18418011.6
Enhanced Enclosure                                                 12284982.8
Basic Enclosure                                                    10795361.9
Mininum Efficiency Heat Pump with Electric Heat Backup              7968124.7
Mininum Efficiency Heat Pump with Existing Heat Backup              7892844.5
Mininum Efficiency Whole Home Electrification                       2836823.8
Heat Pump Water Heater                                              2197122.7
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4725350.6
Basic Enclosure + High Efficiency Whole Home Electrification       4627449.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4524932.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4427030.7
High Efficiency Whole Home Electrification                         4277823.5
High Efficiency Heat Pump with Electric Heat Backup                2914582.6
Enhanced Enclosure                                                 1213875.7
Mininum Efficiency Heat Pump with Electric Heat Backup             1171392.2
Mininum Efficiency Heat Pump with Existing Heat Backup             1075844.6
Basic Enclosure                                                     944161.2
Heat Pump Water Heater                                              897831.9
Heat Pump Clothes Dryer                                             180725.2
Mininum Efficiency Whole Home Electrification                  

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    11266393.1
Basic Enclosure + High Efficiency Whole Home Electrification       11107509.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     10839134.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        10680251.0
High Efficiency Whole Home Electrification                         10303948.7
High Efficiency Heat Pump with Electric Heat Backup                 7726141.4
Mininum Efficiency Heat Pump with Electric Heat Backup              3556608.3
Mininum Efficiency Heat Pump with Existing Heat Backup              3500547.5
Enhanced Enclosure                                                  3027149.7
Basic Enclosure                                                     2411243.6
Heat Pump Water Heater                                              1644689.4
Mininum Efficiency Whole Home Electrification                        445317.0
Heat Pump Clothes Dryer                            

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    6246888.9
Basic Enclosure + High Efficiency Whole Home Electrification       5953180.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     5926993.2
High Efficiency Whole Home Electrification                         5689528.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        5633284.5
High Efficiency Heat Pump with Electric Heat Backup                3172325.1
Heat Pump Water Heater                                             1833698.6
Enhanced Enclosure                                                 1230561.0
Mininum Efficiency Heat Pump with Electric Heat Backup             1081478.8
Mininum Efficiency Heat Pump with Existing Heat Backup             1077547.6
Basic Enclosure                                                     713828.6
Heat Pump Clothes Dryer                                             284036.9
Mininum Efficiency Whole Home Electrification                  

### Annual Total Energy Savings

In [26]:
indicator ='saving_energy.total_mmbtu'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_energy_total_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_total_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_total_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_total_mmbtu'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1052226.1
Basic Enclosure + High Efficiency Whole Home Electrification       1041875.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1023331.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1012980.6
High Efficiency Whole Home Electrification                          943942.5
High Efficiency Heat Pump with Electric Heat Backup                 753078.8
Mininum Efficiency Heat Pump with Electric Heat Backup              481931.4
Enhanced Enclosure                                                  466031.5
Mininum Efficiency Whole Home Electrification                       446293.2
Basic Enclosure                                                     408266.9
Mininum Efficiency Heat Pump with Existing Heat Backup              357123.5
Heat Pump Water Heater                                              120829.5
Heat Pump Clothes Dryer                                        

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    129998.3
Basic Enclosure + High Efficiency Whole Home Electrification       127458.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     122110.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        119570.0
High Efficiency Whole Home Electrification                         118386.2
High Efficiency Heat Pump with Electric Heat Backup                 77987.0
Mininum Efficiency Heat Pump with Electric Heat Backup              32755.4
Enhanced Enclosure                                                  32686.4
Mininum Efficiency Heat Pump with Existing Heat Backup              28736.9
Basic Enclosure                                                     25685.5
Heat Pump Water Heater                                              25347.8
Mininum Efficiency Whole Home Electrification                        8903.1
Heat Pump Clothes Dryer                                              4785.4

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    342329.8
Basic Enclosure + High Efficiency Whole Home Electrification       338207.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     326886.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        322763.9
High Efficiency Whole Home Electrification                         317356.8
High Efficiency Heat Pump with Electric Heat Backup                221721.1
Mininum Efficiency Heat Pump with Electric Heat Backup             113537.7
Mininum Efficiency Heat Pump with Existing Heat Backup              99546.8
Enhanced Enclosure                                                  85774.4
Basic Enclosure                                                     68597.1
Heat Pump Water Heater                                              64262.3
Mininum Efficiency Whole Home Electrification                       61546.6
Heat Pump Clothes Dryer                                             10912.4

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    171860.2
Basic Enclosure + High Efficiency Whole Home Electrification       164239.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     160008.1
High Efficiency Whole Home Electrification                         157398.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        152387.1
High Efficiency Heat Pump with Electric Heat Backup                 83808.1
Heat Pump Water Heater                                              52063.0
Enhanced Enclosure                                                  32661.3
Mininum Efficiency Heat Pump with Electric Heat Backup              29554.8
Mininum Efficiency Heat Pump with Existing Heat Backup              28486.8
Basic Enclosure                                                     18931.3
Mininum Efficiency Whole Home Electrification                       12944.3
Heat Pump Clothes Dryer                                              7751.7

### Annual Total Emission Savings

In [27]:
indicator ='saving_emission.total_kgCO2e'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_total_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_total_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_total_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_total_kgCO2e'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    76173212.7
Basic Enclosure + High Efficiency Whole Home Electrification       75203493.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     74227341.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        73257621.9
High Efficiency Whole Home Electrification                         67413737.9
High Efficiency Heat Pump with Electric Heat Backup                54291767.1
Enhanced Enclosure                                                 34090010.1
Mininum Efficiency Heat Pump with Electric Heat Backup             32484213.7
Basic Enclosure                                                    29648605.7
Mininum Efficiency Whole Home Electrification                      26729391.6
Mininum Efficiency Heat Pump with Existing Heat Backup             26112702.6
Heat Pump Water Heater                                              8244572.4
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    10361105.6
Basic Enclosure + High Efficiency Whole Home Electrification       10136507.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup      9814933.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup         9590335.3
High Efficiency Whole Home Electrification                          9382032.1
High Efficiency Heat Pump with Electric Heat Backup                 6441364.3
Mininum Efficiency Heat Pump with Electric Heat Backup              2820119.4
Enhanced Enclosure                                                  2744859.9
Mininum Efficiency Heat Pump with Existing Heat Backup              2545885.4
Basic Enclosure                                                     2126492.5
Heat Pump Water Heater                                              1791610.4
Mininum Efficiency Whole Home Electrification                        558256.7
Heat Pump Clothes Dryer                            

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    26570069.6
Basic Enclosure + High Efficiency Whole Home Electrification       26189495.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     25512433.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        25131859.4
High Efficiency Whole Home Electrification                         24523487.2
High Efficiency Heat Pump with Electric Heat Backup                17806323.3
Mininum Efficiency Heat Pump with Electric Heat Backup              9104516.9
Mininum Efficiency Heat Pump with Existing Heat Backup              8356276.3
Enhanced Enclosure                                                  6897441.1
Basic Enclosure                                                     5438108.6
Heat Pump Water Heater                                              4362202.0
Mininum Efficiency Whole Home Electrification                       3820629.4
Heat Pump Clothes Dryer                            

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    13211880.7
Basic Enclosure + High Efficiency Whole Home Electrification       12712240.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     12395224.3
High Efficiency Whole Home Electrification                         12121777.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        11895584.1
High Efficiency Heat Pump with Electric Heat Backup                 6803931.8
Heat Pump Water Heater                                              3689406.3
Enhanced Enclosure                                                  2547735.2
Mininum Efficiency Heat Pump with Electric Heat Backup              2493662.4
Mininum Efficiency Heat Pump with Existing Heat Backup              2432780.5
Basic Enclosure                                                     1595398.3
Mininum Efficiency Whole Home Electrification                        839122.1
Heat Pump Clothes Dryer                            

## 2.4 Annual Savings Per Dwelling Unit

### Annual Bill Savings Per Dwelling Unit

In [28]:
indicator ='saving_cbill.total_usd'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_mean_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_mean_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_mean_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_mean_usd'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1491.8
Basic Enclosure + High Efficiency Whole Home Electrification       1469.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1451.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1428.7
High Efficiency Whole Home Electrification                         1254.1
High Efficiency Heat Pump with Electric Heat Backup                1049.2
Enhanced Enclosure                                                  699.8
Basic Enclosure                                                     614.9
Mininum Efficiency Heat Pump with Electric Heat Backup              453.9
Mininum Efficiency Heat Pump with Existing Heat Backup              449.6
Mininum Efficiency Whole Home Electrification                       248.1
Heat Pump Water Heater                                              125.5
Heat Pump Clothes Dryer                                              37.3
Induction Cooking        

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    585.6
Basic Enclosure + High Efficiency Whole Home Electrification       573.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     560.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        548.6
High Efficiency Whole Home Electrification                         530.1
High Efficiency Heat Pump with Electric Heat Backup                361.2
Enhanced Enclosure                                                 150.4
Mininum Efficiency Heat Pump with Electric Heat Backup             145.2
Mininum Efficiency Heat Pump with Existing Heat Backup             139.5
Basic Enclosure                                                    118.9
Heat Pump Water Heater                                             112.7
Heat Pump Clothes Dryer                                             29.8
Mininum Efficiency Whole Home Electrification                       25.8
Induction Cooking                     

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1139.5
Basic Enclosure + High Efficiency Whole Home Electrification       1123.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1096.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1080.2
High Efficiency Whole Home Electrification                         1042.1
High Efficiency Heat Pump with Electric Heat Backup                 781.4
Mininum Efficiency Heat Pump with Electric Heat Backup              359.7
Mininum Efficiency Heat Pump with Existing Heat Backup              354.0
Enhanced Enclosure                                                  307.5
Basic Enclosure                                                     244.9
Heat Pump Water Heater                                              167.4
Mininum Efficiency Whole Home Electrification                       116.8
Heat Pump Clothes Dryer                                              42.8
Induction Cooking        

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    520.9
Basic Enclosure + High Efficiency Whole Home Electrification       496.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     494.2
High Efficiency Whole Home Electrification                         474.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        469.7
High Efficiency Heat Pump with Electric Heat Backup                265.2
Heat Pump Water Heater                                             154.7
Enhanced Enclosure                                                 109.2
Mininum Efficiency Heat Pump with Existing Heat Backup              91.8
Mininum Efficiency Heat Pump with Electric Heat Backup              90.4
Basic Enclosure                                                     67.1
Mininum Efficiency Whole Home Electrification                       43.1
Heat Pump Clothes Dryer                                             31.5
Induction Cooking                     

### Annual Energy Savings Per Dwelling Unit

In [29]:
indicator ='saving_energy.total_mmbtu'
results = segment_mean_savings(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_mean_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_mean_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_mean_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_mean_mmbtu'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    59.9
Basic Enclosure + High Efficiency Whole Home Electrification       59.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     58.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        57.7
High Efficiency Whole Home Electrification                         53.8
High Efficiency Heat Pump with Electric Heat Backup                42.9
Mininum Efficiency Whole Home Electrification                      39.0
Mininum Efficiency Heat Pump with Electric Heat Backup             27.5
Enhanced Enclosure                                                 26.5
Basic Enclosure                                                    23.3
Mininum Efficiency Heat Pump with Existing Heat Backup             20.3
Heat Pump Water Heater                                              6.9
Electric Cooking                                                    1.7
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    16.1
Basic Enclosure + High Efficiency Whole Home Electrification       15.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     15.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        14.8
High Efficiency Whole Home Electrification                         14.7
High Efficiency Heat Pump with Electric Heat Backup                 9.7
Mininum Efficiency Heat Pump with Electric Heat Backup              4.1
Enhanced Enclosure                                                  4.1
Mininum Efficiency Whole Home Electrification                       3.9
Mininum Efficiency Heat Pump with Existing Heat Backup              3.7
Basic Enclosure                                                     3.2
Heat Pump Water Heater                                              3.2
Electric Cooking                                                    1.5
Heat Pump Clothes Dryer                            

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    34.6
Basic Enclosure + High Efficiency Whole Home Electrification       34.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     33.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        32.6
High Efficiency Whole Home Electrification                         32.1
High Efficiency Heat Pump with Electric Heat Backup                22.4
Mininum Efficiency Whole Home Electrification                      16.1
Mininum Efficiency Heat Pump with Electric Heat Backup             11.5
Mininum Efficiency Heat Pump with Existing Heat Backup             10.1
Enhanced Enclosure                                                  8.7
Basic Enclosure                                                     7.0
Heat Pump Water Heater                                              6.5
Electric Cooking                                                    1.9
Heat Pump Clothes Dryer                            

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    14.3
Basic Enclosure + High Efficiency Whole Home Electrification       13.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     13.3
High Efficiency Whole Home Electrification                         13.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        12.7
High Efficiency Heat Pump with Electric Heat Backup                 7.0
Mininum Efficiency Whole Home Electrification                       4.6
Heat Pump Water Heater                                              4.4
Enhanced Enclosure                                                  2.9
Mininum Efficiency Heat Pump with Electric Heat Backup              2.5
Mininum Efficiency Heat Pump with Existing Heat Backup              2.4
Basic Enclosure                                                     1.8
Electric Cooking                                                    1.5
Heat Pump Clothes Dryer                            

### Annual Percentage Energy Savings Per Dwelling Unit

In [30]:
indicator ='pct_saving_energy.total_%'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_pct_saving_energy_mean_%'] = results[0]
saving_results[f'segment2_{income}_pct_saving_energy_mean_%'] = results[1]
saving_results[f'segment3_{income}_pct_saving_energy_mean_%'] = results[2]
saving_results[f'segment4_{income}_pct_saving_energy_mean_%'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    62.2
Basic Enclosure + High Efficiency Whole Home Electrification       61.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     60.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        59.8
High Efficiency Whole Home Electrification                         55.5
High Efficiency Heat Pump with Electric Heat Backup                44.1
Mininum Efficiency Whole Home Electrification                      35.0
Enhanced Enclosure                                                 27.3
Mininum Efficiency Heat Pump with Electric Heat Backup             27.0
Basic Enclosure                                                    23.8
Mininum Efficiency Heat Pump with Existing Heat Backup             20.8
Heat Pump Water Heater                                              7.4
Electric Cooking                                                    1.7
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    43.7
Basic Enclosure + High Efficiency Whole Home Electrification       42.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     40.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        40.1
High Efficiency Whole Home Electrification                         39.9
High Efficiency Heat Pump with Electric Heat Backup                25.8
Enhanced Enclosure                                                 10.7
Mininum Efficiency Heat Pump with Electric Heat Backup             10.4
Mininum Efficiency Heat Pump with Existing Heat Backup              9.6
Mininum Efficiency Whole Home Electrification                       9.2
Heat Pump Water Heater                                              8.9
Basic Enclosure                                                     8.5
Electric Cooking                                                    4.5
Heat Pump Clothes Dryer                            

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    47.0
Basic Enclosure + High Efficiency Whole Home Electrification       46.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     44.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        44.4
High Efficiency Whole Home Electrification                         43.6
High Efficiency Heat Pump with Electric Heat Backup                30.5
Mininum Efficiency Whole Home Electrification                      20.5
Mininum Efficiency Heat Pump with Electric Heat Backup             15.3
Mininum Efficiency Heat Pump with Existing Heat Backup             13.4
Enhanced Enclosure                                                 11.6
Basic Enclosure                                                     9.6
Heat Pump Water Heater                                              8.8
Electric Cooking                                                    2.6
Heat Pump Clothes Dryer                            

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    39.2
Basic Enclosure + High Efficiency Whole Home Electrification       38.1
High Efficiency Whole Home Electrification                         36.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     36.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        35.2
High Efficiency Heat Pump with Electric Heat Backup                19.3
Mininum Efficiency Whole Home Electrification                      12.9
Heat Pump Water Heater                                             12.2
Enhanced Enclosure                                                  6.9
Mininum Efficiency Heat Pump with Electric Heat Backup              6.3
Mininum Efficiency Heat Pump with Existing Heat Backup              6.1
Basic Enclosure                                                     5.0
Electric Cooking                                                    4.7
Heat Pump Clothes Dryer                            

### Annual Emission Savings Per Dwelling Unit

In [31]:
indicator ='saving_emission.total_kgCO2e'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_mean_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_mean_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_mean_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_mean_kgCO2e'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4339.1
Basic Enclosure + High Efficiency Whole Home Electrification       4283.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4228.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4173.0
High Efficiency Whole Home Electrification                         3840.1
High Efficiency Heat Pump with Electric Heat Backup                3092.6
Mininum Efficiency Whole Home Electrification                      2337.8
Enhanced Enclosure                                                 1941.9
Mininum Efficiency Heat Pump with Electric Heat Backup             1850.4
Basic Enclosure                                                    1688.9
Mininum Efficiency Heat Pump with Existing Heat Backup             1487.5
Heat Pump Water Heater                                              471.0
Electric Cooking                                                    110.4
Heat Pump Clothes Dryer  

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1284.0
Basic Enclosure + High Efficiency Whole Home Electrification       1256.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1216.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1188.5
High Efficiency Whole Home Electrification                         1162.7
High Efficiency Heat Pump with Electric Heat Backup                 798.3
Mininum Efficiency Heat Pump with Electric Heat Backup              349.5
Enhanced Enclosure                                                  340.2
Mininum Efficiency Heat Pump with Existing Heat Backup              330.2
Basic Enclosure                                                     267.8
Mininum Efficiency Whole Home Electrification                       245.9
Heat Pump Water Heater                                              224.9
Electric Cooking                                                    104.7
Heat Pump Clothes Dryer  

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    2687.3
Basic Enclosure + High Efficiency Whole Home Electrification       2648.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     2580.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        2541.8
High Efficiency Whole Home Electrification                         2480.3
High Efficiency Heat Pump with Electric Heat Backup                1800.9
Mininum Efficiency Whole Home Electrification                      1002.3
Mininum Efficiency Heat Pump with Electric Heat Backup              920.8
Mininum Efficiency Heat Pump with Existing Heat Backup              845.1
Enhanced Enclosure                                                  700.6
Basic Enclosure                                                     552.4
Heat Pump Water Heater                                              444.1
Electric Cooking                                                    125.2
Heat Pump Clothes Dryer  

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1101.6
Basic Enclosure + High Efficiency Whole Home Electrification       1060.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1033.5
High Efficiency Whole Home Electrification                         1010.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup         991.9
High Efficiency Heat Pump with Electric Heat Backup                 568.9
Heat Pump Water Heater                                              311.3
Mininum Efficiency Whole Home Electrification                       295.6
Enhanced Enclosure                                                  226.1
Mininum Efficiency Heat Pump with Electric Heat Backup              208.5
Mininum Efficiency Heat Pump with Existing Heat Backup              207.3
Basic Enclosure                                                     150.1
Electric Cooking                                                    104.7
Electric Clothes Dryer   

## 2.5. Pecent Stock with Positive and Negtive Savings

### Segment 1

In [32]:
segment = 'segment1'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment1_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment1_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment1_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 2

In [33]:
segment = 'segment2'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment2_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment2_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment2_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 3

In [34]:
segment = 'segment3'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment3_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment3_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment3_{income}_pct_energy_positive_savings_%'] = results[2]*100

### Segment 4

In [35]:
segment = 'segment4'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment4_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment4_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment4_{income}_pct_energy_positive_savings_%'] = results[2]*100

## 2.6. Average Energy Burden (%) Before and after Upgrade

In [36]:
results = segment_mean_energy_burden('post-upgrade_energy_burden_2023_cbills.%', community_results) 
saving_results[[f'segment1_{income}_energy_burden_before_upgrade_%',f'segment1_{income}_energy_burden_after_upgrade_%']] = results[0]
saving_results[[f'segment2_{income}_energy_burden_before_upgrade_%',f'segment2_{income}_energy_burden_after_upgrade_%']] = results[1]
saving_results[[f'segment3_{income}_energy_burden_before_upgrade_%',f'segment3_{income}_energy_burden_after_upgrade_%']] = results[2]
saving_results[[f'segment4_{income}_energy_burden_before_upgrade_%',f'segment4_{income}_energy_burden_after_upgrade_%']] = results[3]

energy burden (%) before and after upgrade
segment1


/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/3555432078.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/3555432078.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/3555432078.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,14.9,6.9
Basic Enclosure + High Efficiency Whole Home Electrification,14.9,7.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,14.9,7.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,14.9,7.2
High Efficiency Whole Home Electrification,14.9,8.1
High Efficiency Heat Pump with Electric Heat Backup,14.9,9.1
Enhanced Enclosure,14.9,11.1
Basic Enclosure,14.9,11.6
Mininum Efficiency Heat Pump with Electric Heat Backup,14.9,12.2


segment2


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,12.9,7.3
Basic Enclosure + High Efficiency Whole Home Electrification,12.9,7.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,12.9,7.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,12.9,7.6
High Efficiency Whole Home Electrification,12.9,7.8
High Efficiency Heat Pump with Electric Heat Backup,12.9,9.4
Mininum Efficiency Heat Pump with Existing Heat Backup,13.1,11.5
Mininum Efficiency Heat Pump with Electric Heat Backup,12.9,11.3
Enhanced Enclosure,12.9,11.5


segment3


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,14.1,7.8
Basic Enclosure + High Efficiency Whole Home Electrification,14.1,7.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,14.1,8.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,14.1,8.1
High Efficiency Whole Home Electrification,14.1,8.2
High Efficiency Heat Pump with Electric Heat Backup,14.1,9.8
Mininum Efficiency Heat Pump with Electric Heat Backup,14.1,11.9
Mininum Efficiency Heat Pump with Existing Heat Backup,14.1,11.9
Enhanced Enclosure,14.1,12.7


segment4


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,10.3,6.3
Basic Enclosure + High Efficiency Whole Home Electrification,10.3,6.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,10.3,6.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,10.3,6.6
High Efficiency Whole Home Electrification,10.3,6.6
High Efficiency Heat Pump with Electric Heat Backup,10.3,8.0
Heat Pump Water Heater,10.3,9.3
Enhanced Enclosure,10.5,9.6
Mininum Efficiency Heat Pump with Electric Heat Backup,10.3,9.4


# 3. Results for Segments (0-80% AMI)

Segment Definition

Segment 1. Occupied, Single family detached, 1940-1979, 0-80% AMI  

Segment 2. Occupied, Multifamily with 5+ units, 1940-1979, 0-80% AMI 

Segment 3. Occupied, Single family detached, 1980+, 0-80% AMI 

Segment 4. Occupied, Multifamily with 5+ units, 1980+, 0-80% AMI  

In [37]:
community_results = pd.read_parquet(f"data_/community_building_samples_with_upgrade_cost_and_bill/{community}/processed_upgrade_results.parquet")

income = '0-80%AMI'
vacancy_status = ['Occupied']

seg1_building_type = ['Single-Family Detached']
seg1_vintage = ['1940s', '1950s', '1960s', '1970s']
seg1_median_income = ['0-30%','30-60%','60-80%']

seg2_building_type = ['Multi-Family with 5+ Units']
seg2_vintage = ['1940s', '1950s', '1960s', '1970s']
seg2_median_income = ['0-30%','30-60%','60-80%']

seg3_building_type = ['Single-Family Detached']
seg3_vintage = ['1980s', '1990s', '2000s', '2010s']
seg3_median_income = ['0-30%','30-60%','60-80%']

seg4_building_type = ['Multi-Family with 5+ Units']
seg4_vintage = ['1980s', '1990s', '2000s', '2010s']
seg4_median_income = ['0-30%','30-60%','60-80%']

community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg1_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg1_vintage) &  
                     community_results['build_existing_model.area_median_income'].isin(seg1_median_income),
                     'segment'] = 'segment1'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg2_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg2_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg2_median_income),
                     'segment'] = 'segment2'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg3_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg3_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg3_median_income),
                     'segment'] = 'segment3'
community_results.loc[community_results['build_existing_model.vacancy_status'].isin(vacancy_status) & 
                     community_results['build_existing_model.geometry_building_type_recs'].isin(seg4_building_type) & 
                     community_results['build_existing_model.vintage'].isin(seg4_vintage) & 
                     community_results['build_existing_model.area_median_income'].isin(seg4_median_income),
                     'segment'] = 'segment4'

## 3.1 Upgrade Cost Per Dwelling Unit 

In [38]:
results = segment_mean_upgrade_cost(community_results) 
saving_results[f'segment1_{income}_upgrade_cost_mean_usd'] = results[0]
saving_results[f'segment2_{income}_upgrade_cost_mean_usd'] = results[1]
saving_results[f'segment3_{income}_upgrade_cost_mean_usd'] = results[2]
saving_results[f'segment4_{income}_upgrade_cost_mean_usd'] = results[3]

segment1: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1115.8
Electric Clothes Dryer                                              1253.0
Induction Cooking                                                   1918.7
Heat Pump Clothes Dryer                                             2521.5
Heat Pump Water Heater                                              3120.6
Basic Enclosure                                                     6615.8
Enhanced Enclosure                                                  9533.9
Mininum Efficiency Whole Home Electrification                      15133.9
Mininum Efficiency Heat Pump with Existing Heat Backup             16673.8
Mininum Efficiency Heat Pump with Electric Heat Backup             16857.3
High Efficiency Heat Pump with Electric Heat Backup                28911.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        33970.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     35923.5
High Efficie

segment2: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1115.8
Electric Clothes Dryer                                              1253.0
Basic Enclosure                                                     1457.8
Induction Cooking                                                   1918.7
Enhanced Enclosure                                                  2325.1
Heat Pump Clothes Dryer                                             2521.5
Heat Pump Water Heater                                              2875.4
Mininum Efficiency Whole Home Electrification                       3347.6
Mininum Efficiency Heat Pump with Electric Heat Backup              9148.3
Mininum Efficiency Heat Pump with Existing Heat Backup              9165.1
High Efficiency Heat Pump with Electric Heat Backup                21854.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        25143.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     25778.4
High Efficie

segment3: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1115.8
Induction Cooking                                                   1918.7
Heat Pump Clothes Dryer                                             2521.5
Heat Pump Water Heater                                              3494.4
Basic Enclosure                                                     3952.2
Enhanced Enclosure                                                  6710.1
Mininum Efficiency Whole Home Electrification                       7844.4
Mininum Efficiency Heat Pump with Existing Heat Backup             13282.5
Mininum Efficiency Heat Pump with Electric Heat Backup             13339.0
High Efficiency Heat Pump with Electric Heat Backup                27808.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        33511.6
High Efficiency Whole Home Electrification                         35413.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     35457.2
Basic Enclos

segment4: upgrade cost (usd) per dwelling unit


upgrade_name
Electric Cooking                                                    1115.8
Basic Enclosure                                                     1211.7
Electric Clothes Dryer                                              1253.0
Induction Cooking                                                   1918.7
Enhanced Enclosure                                                  2019.0
Mininum Efficiency Whole Home Electrification                       2213.2
Heat Pump Clothes Dryer                                             2521.5
Heat Pump Water Heater                                              2879.8
Mininum Efficiency Heat Pump with Existing Heat Backup              8269.3
Mininum Efficiency Heat Pump with Electric Heat Backup              8286.9
High Efficiency Heat Pump with Electric Heat Backup                22107.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        25393.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     26046.3
High Efficie

 ## 3.2 Annual Savings Per Dollar Invested

### Annual Bill Savings Per Dollar Invested

In [39]:
indicator ='saving_cbill.total_usd'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_bill_per_dollar_invested_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_per_dollar_invested_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_per_dollar_invested_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_per_dollar_invested_usd'] = results[3]

segment1


upgrade_name
Basic Enclosure                                                    0.08997
Enhanced Enclosure                                                 0.07221
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.04170
Heat Pump Water Heater                                             0.04033
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.04013
Basic Enclosure + High Efficiency Whole Home Electrification       0.03801
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.03679
High Efficiency Heat Pump with Electric Heat Backup                0.03631
High Efficiency Whole Home Electrification                         0.03456
Mininum Efficiency Heat Pump with Electric Heat Backup             0.02880
Mininum Efficiency Heat Pump with Existing Heat Backup             0.02826
Mininum Efficiency Whole Home Electrification                      0.02026
Heat Pump Clothes Dryer                                            0.01395
Induction Co

segment2


upgrade_name
Basic Enclosure                                                    0.08354
Enhanced Enclosure                                                 0.06512
Heat Pump Water Heater                                             0.03766
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.02188
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.02176
Basic Enclosure + High Efficiency Whole Home Electrification       0.01987
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.01981
High Efficiency Whole Home Electrification                         0.01864
High Efficiency Heat Pump with Electric Heat Backup                0.01681
Mininum Efficiency Heat Pump with Electric Heat Backup             0.01636
Mininum Efficiency Heat Pump with Existing Heat Backup             0.01562
Heat Pump Clothes Dryer                                            0.01180
Mininum Efficiency Whole Home Electrification                      0.00894
Induction Co

segment3


upgrade_name
Basic Enclosure                                                    0.05617
Heat Pump Water Heater                                             0.04974
Enhanced Enclosure                                                 0.04536
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.03207
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.03089
Basic Enclosure + High Efficiency Whole Home Electrification       0.02975
High Efficiency Whole Home Electrification                         0.02954
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.02880
High Efficiency Heat Pump with Electric Heat Backup                0.02808
Mininum Efficiency Heat Pump with Electric Heat Backup             0.02800
Mininum Efficiency Heat Pump with Existing Heat Backup             0.02776
Heat Pump Clothes Dryer                                            0.01940
Mininum Efficiency Whole Home Electrification                      0.01171
Induction Co

segment4


upgrade_name
Basic Enclosure                                                    0.05827
Enhanced Enclosure                                                 0.05480
Heat Pump Water Heater                                             0.04867
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.01864
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.01818
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.01713
Basic Enclosure + High Efficiency Whole Home Electrification       0.01669
Mininum Efficiency Whole Home Electrification                      0.01621
High Efficiency Whole Home Electrification                         0.01618
High Efficiency Heat Pump with Electric Heat Backup                0.01219
Heat Pump Clothes Dryer                                            0.01185
Mininum Efficiency Heat Pump with Existing Heat Backup             0.01130
Mininum Efficiency Heat Pump with Electric Heat Backup             0.01101
Induction Co

 ### Annual Energy Savings Per Dollar Invested

In [40]:
indicator = 'saving_energy.total_mmbtu'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_per_dollar_invested_mmbtu'] = results[3]

segment1


upgrade_name
Basic Enclosure                                                    0.00330
Enhanced Enclosure                                                 0.00264
Mininum Efficiency Whole Home Electrification                      0.00254
Heat Pump Water Heater                                             0.00181
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00159
Electric Cooking                                                   0.00158
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00156
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00152
Basic Enclosure + High Efficiency Whole Home Electrification       0.00145
High Efficiency Heat Pump with Electric Heat Backup                0.00142
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00140
High Efficiency Whole Home Electrification                         0.00139
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00116
Heat Pump Cl

segment2


upgrade_name
Basic Enclosure                                                    0.00229
Enhanced Enclosure                                                 0.00177
Electric Cooking                                                   0.00138
Mininum Efficiency Whole Home Electrification                      0.00138
Heat Pump Water Heater                                             0.00109
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00060
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00059
Basic Enclosure + High Efficiency Whole Home Electrification       0.00055
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00055
High Efficiency Whole Home Electrification                         0.00052
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00046
High Efficiency Heat Pump with Electric Heat Backup                0.00045
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00042
Heat Pump Cl

segment3


upgrade_name
Heat Pump Water Heater                                             0.00199
Electric Cooking                                                   0.00182
Mininum Efficiency Whole Home Electrification                      0.00160
Basic Enclosure                                                    0.00153
Enhanced Enclosure                                                 0.00123
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00094
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00091
High Efficiency Whole Home Electrification                         0.00089
Basic Enclosure + High Efficiency Whole Home Electrification       0.00088
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00085
Mininum Efficiency Heat Pump with Electric Heat Backup             0.00082
High Efficiency Heat Pump with Electric Heat Backup                0.00077
Mininum Efficiency Heat Pump with Existing Heat Backup             0.00074
Heat Pump Cl

segment4


upgrade_name
Mininum Efficiency Whole Home Electrification                      0.00181
Basic Enclosure                                                    0.00153
Enhanced Enclosure                                                 0.00143
Heat Pump Water Heater                                             0.00142
Electric Cooking                                                   0.00134
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.00050
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.00049
Electric Clothes Dryer                                             0.00049
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.00047
Basic Enclosure + High Efficiency Whole Home Electrification       0.00046
High Efficiency Whole Home Electrification                         0.00045
Heat Pump Clothes Dryer                                            0.00033
High Efficiency Heat Pump with Electric Heat Backup                0.00032
Mininum Effi

 ### Annual Emission Savings Per Dollar Invested

In [41]:
indicator ='saving_emission.total_kgCO2e'
results = segment_savings_per_dollar(indicator, community_results) 
saving_results[f'segment1_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_per_dollar_invested_kgCO2e'] = results[3]

segment1


upgrade_name
Basic Enclosure                                                    0.24196
Enhanced Enclosure                                                 0.19569
Mininum Efficiency Whole Home Electrification                      0.15118
Heat Pump Water Heater                                             0.12284
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.11610
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.11147
Mininum Efficiency Heat Pump with Electric Heat Backup             0.10684
Basic Enclosure + High Efficiency Whole Home Electrification       0.10573
High Efficiency Heat Pump with Electric Heat Backup                0.10327
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.10210
High Efficiency Whole Home Electrification                         0.10004
Electric Cooking                                                   0.09937
Mininum Efficiency Heat Pump with Existing Heat Backup             0.08702
Heat Pump Cl

segment2


upgrade_name
Basic Enclosure                                                    0.18879
Enhanced Enclosure                                                 0.14732
Electric Cooking                                                   0.09840
Mininum Efficiency Whole Home Electrification                      0.08684
Heat Pump Water Heater                                             0.07606
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.04769
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.04746
Basic Enclosure + High Efficiency Whole Home Electrification       0.04377
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.04365
High Efficiency Whole Home Electrification                         0.04112
Mininum Efficiency Heat Pump with Electric Heat Backup             0.03994
Mininum Efficiency Heat Pump with Existing Heat Backup             0.03741
High Efficiency Heat Pump with Electric Heat Backup                0.03730
Heat Pump Cl

segment3


upgrade_name
Heat Pump Water Heater                                             0.13756
Electric Cooking                                                   0.12208
Basic Enclosure                                                    0.12132
Mininum Efficiency Whole Home Electrification                      0.10527
Enhanced Enclosure                                                 0.09947
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.07400
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.07130
High Efficiency Whole Home Electrification                         0.06895
Basic Enclosure + High Efficiency Whole Home Electrification       0.06889
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.06671
Mininum Efficiency Heat Pump with Electric Heat Backup             0.06660
Mininum Efficiency Heat Pump with Existing Heat Backup             0.06256
High Efficiency Heat Pump with Electric Heat Backup                0.06244
Heat Pump Cl

segment4


upgrade_name
Basic Enclosure                                                    0.12953
Mininum Efficiency Whole Home Electrification                      0.11805
Enhanced Enclosure                                                 0.11256
Heat Pump Water Heater                                             0.09984
Electric Cooking                                                   0.08998
Electric Clothes Dryer                                             0.05347
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     0.03916
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        0.03858
Enhanced Enclosure + High Efficiency Whole Home Electrification    0.03643
Basic Enclosure + High Efficiency Whole Home Electrification       0.03586
High Efficiency Whole Home Electrification                         0.03470
High Efficiency Heat Pump with Electric Heat Backup                0.02605
Mininum Efficiency Heat Pump with Existing Heat Backup             0.02572
Mininum Effi

## 3.3 Annual Total Savings

### Annual Total Bill Savings

In [42]:
indicator ='saving_cbill.total_usd'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_total_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_total_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_total_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_total_usd'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    16023986.0
Basic Enclosure + High Efficiency Whole Home Electrification       15751989.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     15607339.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        15335343.1
High Efficiency Whole Home Electrification                         13590842.8
High Efficiency Heat Pump with Electric Heat Backup                11366631.3
Enhanced Enclosure                                                  7453144.9
Basic Enclosure                                                     6444547.4
Mininum Efficiency Heat Pump with Electric Heat Backup              5255587.9
Mininum Efficiency Heat Pump with Existing Heat Backup              5100764.9
Mininum Efficiency Whole Home Electrification                       1945317.8
Heat Pump Water Heater                                              1355962.6
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    3915028.1
Basic Enclosure + High Efficiency Whole Home Electrification       3842204.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     3759261.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3686438.3
High Efficiency Whole Home Electrification                         3546195.6
High Efficiency Heat Pump with Electric Heat Backup                2461430.9
Enhanced Enclosure                                                 1014704.4
Mininum Efficiency Heat Pump with Electric Heat Backup             1003093.2
Mininum Efficiency Heat Pump with Existing Heat Backup              907585.4
Basic Enclosure                                                     800418.5
Heat Pump Water Heater                                              714603.8
Heat Pump Clothes Dryer                                             142959.7
Mininum Efficiency Whole Home Electrification                  

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    5532625.7
Basic Enclosure + High Efficiency Whole Home Electrification       5434357.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     5312406.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        5214137.7
High Efficiency Whole Home Electrification                         5074423.0
High Efficiency Heat Pump with Electric Heat Backup                3788389.4
Mininum Efficiency Heat Pump with Electric Heat Backup             1811453.6
Mininum Efficiency Heat Pump with Existing Heat Backup             1788674.0
Enhanced Enclosure                                                 1463256.1
Basic Enclosure                                                    1067411.0
Heat Pump Water Heater                                              835716.1
Heat Pump Clothes Dryer                                             209146.9
Mininum Efficiency Whole Home Electrification                  

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4804439.1
Basic Enclosure + High Efficiency Whole Home Electrification       4579877.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4563169.8
High Efficiency Whole Home Electrification                         4360929.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        4338607.9
High Efficiency Heat Pump with Electric Heat Backup                2523641.6
Heat Pump Water Heater                                             1297602.2
Enhanced Enclosure                                                  999602.0
Mininum Efficiency Heat Pump with Electric Heat Backup              854689.1
Mininum Efficiency Heat Pump with Existing Heat Backup              854232.7
Basic Enclosure                                                     597583.7
Heat Pump Clothes Dryer                                             210912.8
Mininum Efficiency Whole Home Electrification                  

### Annual Total Energy Savings

In [43]:
indicator ='saving_energy.total_mmbtu'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_energy_total_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_total_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_total_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_total_mmbtu'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    608963.4
Basic Enclosure + High Efficiency Whole Home Electrification       601905.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     592331.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        585273.4
High Efficiency Whole Home Electrification                         545828.7
High Efficiency Heat Pump with Electric Heat Backup                443298.1
Mininum Efficiency Heat Pump with Electric Heat Backup             284730.9
Enhanced Enclosure                                                 272987.2
Mininum Efficiency Whole Home Electrification                      243652.4
Basic Enclosure                                                    236541.2
Mininum Efficiency Heat Pump with Existing Heat Backup             210037.6
Heat Pump Water Heater                                              60877.8
Heat Pump Clothes Dryer                                             10707.4

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    108342.9
Basic Enclosure + High Efficiency Whole Home Electrification       106453.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     102242.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        100353.2
High Efficiency Whole Home Electrification                          98772.8
High Efficiency Heat Pump with Electric Heat Backup                 66228.7
Mininum Efficiency Heat Pump with Electric Heat Backup              28388.1
Enhanced Enclosure                                                  27518.3
Mininum Efficiency Heat Pump with Existing Heat Backup              24370.6
Basic Enclosure                                                     21955.7
Heat Pump Water Heater                                              20593.2
Mininum Efficiency Whole Home Electrification                        8117.6
Heat Pump Clothes Dryer                                              3805.5

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    164061.5
Basic Enclosure + High Efficiency Whole Home Electrification       161511.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     156024.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        153474.5
High Efficiency Whole Home Electrification                         152172.1
High Efficiency Heat Pump with Electric Heat Backup                104075.3
Mininum Efficiency Heat Pump with Electric Heat Backup              52782.9
Mininum Efficiency Heat Pump with Existing Heat Backup              47844.6
Enhanced Enclosure                                                  39567.5
Heat Pump Water Heater                                              33511.4
Basic Enclosure                                                     29002.5
Mininum Efficiency Whole Home Electrification                       25321.2
Heat Pump Clothes Dryer                                              5426.9

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    132232.3
Basic Enclosure + High Efficiency Whole Home Electrification       126405.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     122979.2
High Efficiency Whole Home Electrification                         120724.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        117152.3
High Efficiency Heat Pump with Electric Heat Backup                 65830.5
Heat Pump Water Heater                                              37782.2
Enhanced Enclosure                                                  26070.9
Mininum Efficiency Heat Pump with Electric Heat Backup              22524.5
Mininum Efficiency Heat Pump with Existing Heat Backup              22237.9
Basic Enclosure                                                     15639.5
Mininum Efficiency Whole Home Electrification                        9863.1
Heat Pump Clothes Dryer                                              5808.5

### Annual Total Emission Savings

In [44]:
indicator ='saving_emission.total_kgCO2e'
results = segment_annual_total_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_total_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_total_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_total_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_total_kgCO2e'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    44473016.8
Basic Enclosure + High Efficiency Whole Home Electrification       43818421.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     43354962.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        42700367.3
High Efficiency Whole Home Electrification                         39346154.3
High Efficiency Heat Pump with Electric Heat Backup                32325117.3
Enhanced Enclosure                                                 20199578.6
Mininum Efficiency Heat Pump with Electric Heat Backup             19499737.1
Basic Enclosure                                                    17331321.3
Mininum Efficiency Heat Pump with Existing Heat Backup             15709644.5
Mininum Efficiency Whole Home Electrification                      14514334.0
Heat Pump Water Heater                                              4130581.8
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    8626311.7
Basic Enclosure + High Efficiency Whole Home Electrification       8462588.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     8199354.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        8035631.2
High Efficiency Whole Home Electrification                         7825410.1
High Efficiency Heat Pump with Electric Heat Backup                5462679.5
Mininum Efficiency Heat Pump with Electric Heat Backup             2448490.8
Enhanced Enclosure                                                 2295504.7
Mininum Efficiency Heat Pump with Existing Heat Backup             2174339.7
Basic Enclosure                                                    1808839.6
Heat Pump Water Heater                                             1443368.7
Mininum Efficiency Whole Home Electrification                       509857.0
Heat Pump Clothes Dryer                                        

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    12815013.3
Basic Enclosure + High Efficiency Whole Home Electrification       12582237.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     12263075.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        12030299.7
High Efficiency Whole Home Electrification                         11843795.1
High Efficiency Heat Pump with Electric Heat Backup                 8422554.4
Mininum Efficiency Heat Pump with Electric Heat Backup              4309440.5
Mininum Efficiency Heat Pump with Existing Heat Backup              4030703.8
Enhanced Enclosure                                                  3208948.1
Heat Pump Water Heater                                              2311129.9
Basic Enclosure                                                     2305249.4
Mininum Efficiency Whole Home Electrification                       1669869.1
Heat Pump Clothes Dryer                            

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    10220162.9
Basic Enclosure + High Efficiency Whole Home Electrification        9840805.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup      9586176.2
High Efficiency Whole Home Electrification                          9350698.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup         9206818.7
High Efficiency Heat Pump with Electric Heat Backup                 5394142.4
Heat Pump Water Heater                                              2662104.6
Enhanced Enclosure                                                  2053132.0
Mininum Efficiency Heat Pump with Electric Heat Backup              1962937.7
Mininum Efficiency Heat Pump with Existing Heat Backup              1943874.6
Basic Enclosure                                                     1328339.9
Mininum Efficiency Whole Home Electrification                        644315.8
Heat Pump Clothes Dryer                            

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/3926536263.py:6: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



## 3.4 Annual Savings Per Dwelling Unit

### Annual Bill Savings Per Dwelling Unit

In [45]:
indicator ='saving_cbill.total_usd'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_bill_mean_usd'] = results[0]
saving_results[f'segment2_{income}_saving_bill_mean_usd'] = results[1]
saving_results[f'segment3_{income}_saving_bill_mean_usd'] = results[2]
saving_results[f'segment4_{income}_saving_bill_mean_usd'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1480.0
Basic Enclosure + High Efficiency Whole Home Electrification       1454.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1441.6
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1416.4
High Efficiency Whole Home Electrification                         1255.3
High Efficiency Heat Pump with Electric Heat Backup                1049.9
Enhanced Enclosure                                                  688.4
Basic Enclosure                                                     595.2
Mininum Efficiency Heat Pump with Electric Heat Backup              485.4
Mininum Efficiency Heat Pump with Existing Heat Backup              471.1
Mininum Efficiency Whole Home Electrification                       306.6
Heat Pump Water Heater                                              125.8
Heat Pump Clothes Dryer                                              35.2
Induction Cooking        

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    584.2
Basic Enclosure + High Efficiency Whole Home Electrification       573.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     561.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        550.1
High Efficiency Whole Home Electrification                         529.2
High Efficiency Heat Pump with Electric Heat Backup                367.3
Enhanced Enclosure                                                 151.4
Mininum Efficiency Heat Pump with Electric Heat Backup             149.7
Mininum Efficiency Heat Pump with Existing Heat Backup             143.1
Basic Enclosure                                                    121.8
Heat Pump Water Heater                                             108.3
Mininum Efficiency Whole Home Electrification                       29.9
Heat Pump Clothes Dryer                                             29.8
Induction Cooking                     

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1140.5
Basic Enclosure + High Efficiency Whole Home Electrification       1120.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1095.1
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1074.9
High Efficiency Whole Home Electrification                         1046.1
High Efficiency Heat Pump with Electric Heat Backup                 781.0
Mininum Efficiency Heat Pump with Electric Heat Backup              373.4
Mininum Efficiency Heat Pump with Existing Heat Backup              368.7
Enhanced Enclosure                                                  304.3
Basic Enclosure                                                     222.0
Heat Pump Water Heater                                              173.8
Mininum Efficiency Whole Home Electrification                        91.8
Heat Pump Clothes Dryer                                              48.9
Induction Cooking        

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    511.2
Basic Enclosure + High Efficiency Whole Home Electrification       487.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     485.5
High Efficiency Whole Home Electrification                         464.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        461.6
High Efficiency Heat Pump with Electric Heat Backup                269.4
Heat Pump Water Heater                                             140.1
Enhanced Enclosure                                                 110.6
Mininum Efficiency Heat Pump with Existing Heat Backup              93.5
Mininum Efficiency Heat Pump with Electric Heat Backup              91.2
Basic Enclosure                                                     70.6
Mininum Efficiency Whole Home Electrification                       35.9
Heat Pump Clothes Dryer                                             29.9
Induction Cooking                     

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/3071005957.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/3071005957.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/3071005957.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

### Annual Energy Savings Per Dwelling Unit

In [46]:
indicator ='saving_energy.total_mmbtu'
results = segment_mean_savings(indicator, community_results) 
saving_results[f'segment1_{income}_saving_energy_mean_mmbtu'] = results[0]
saving_results[f'segment2_{income}_saving_energy_mean_mmbtu'] = results[1]
saving_results[f'segment3_{income}_saving_energy_mean_mmbtu'] = results[2]
saving_results[f'segment4_{income}_saving_energy_mean_mmbtu'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    56.2
Basic Enclosure + High Efficiency Whole Home Electrification       55.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     54.7
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        54.1
High Efficiency Whole Home Electrification                         50.4
High Efficiency Heat Pump with Electric Heat Backup                40.9
Mininum Efficiency Whole Home Electrification                      38.4
Mininum Efficiency Heat Pump with Electric Heat Backup             26.3
Enhanced Enclosure                                                 25.2
Basic Enclosure                                                    21.8
Mininum Efficiency Heat Pump with Existing Heat Backup             19.4
Heat Pump Water Heater                                              5.6
Electric Cooking                                                    1.8
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    16.2
Basic Enclosure + High Efficiency Whole Home Electrification       15.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     15.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        15.0
High Efficiency Whole Home Electrification                         14.7
High Efficiency Heat Pump with Electric Heat Backup                 9.9
Mininum Efficiency Whole Home Electrification                       4.6
Mininum Efficiency Heat Pump with Electric Heat Backup              4.2
Enhanced Enclosure                                                  4.1
Mininum Efficiency Heat Pump with Existing Heat Backup              3.8
Basic Enclosure                                                     3.3
Heat Pump Water Heater                                              3.1
Electric Cooking                                                    1.5
Heat Pump Clothes Dryer                            

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    33.8
Basic Enclosure + High Efficiency Whole Home Electrification       33.3
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     32.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        31.6
High Efficiency Whole Home Electrification                         31.4
High Efficiency Heat Pump with Electric Heat Backup                21.5
Mininum Efficiency Whole Home Electrification                      12.5
Mininum Efficiency Heat Pump with Electric Heat Backup             10.9
Mininum Efficiency Heat Pump with Existing Heat Backup              9.9
Enhanced Enclosure                                                  8.2
Heat Pump Water Heater                                              7.0
Basic Enclosure                                                     6.0
Electric Cooking                                                    2.0
Heat Pump Clothes Dryer                            

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    14.1
Basic Enclosure + High Efficiency Whole Home Electrification       13.4
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     13.1
High Efficiency Whole Home Electrification                         12.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        12.5
High Efficiency Heat Pump with Electric Heat Backup                 7.0
Heat Pump Water Heater                                              4.1
Mininum Efficiency Whole Home Electrification                       4.0
Enhanced Enclosure                                                  2.9
Mininum Efficiency Heat Pump with Existing Heat Backup              2.4
Mininum Efficiency Heat Pump with Electric Heat Backup              2.4
Basic Enclosure                                                     1.8
Electric Cooking                                                    1.5
Heat Pump Clothes Dryer                            

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/396568384.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/396568384.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/396568384.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

### Annual Percentage Energy Savings Per Dwelling Unit

In [47]:
indicator ='pct_saving_energy.total_%'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_pct_saving_energy_mean_%'] = results[0]
saving_results[f'segment2_{income}_pct_saving_energy_mean_%'] = results[1]
saving_results[f'segment3_{income}_pct_saving_energy_mean_%'] = results[2]
saving_results[f'segment4_{income}_pct_saving_energy_mean_%'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    61.5
Basic Enclosure + High Efficiency Whole Home Electrification       60.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     59.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        59.1
High Efficiency Whole Home Electrification                         54.7
High Efficiency Heat Pump with Electric Heat Backup                44.2
Mininum Efficiency Whole Home Electrification                      35.2
Enhanced Enclosure                                                 27.3
Mininum Efficiency Heat Pump with Electric Heat Backup             26.9
Basic Enclosure                                                    23.7
Mininum Efficiency Heat Pump with Existing Heat Backup             20.7
Heat Pump Water Heater                                              6.5
Electric Cooking                                                    1.7
Heat Pump Clothes Dryer                            

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    43.8
Basic Enclosure + High Efficiency Whole Home Electrification       43.1
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     41.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        40.5
High Efficiency Whole Home Electrification                         40.1
High Efficiency Heat Pump with Electric Heat Backup                26.3
Enhanced Enclosure                                                 10.8
Mininum Efficiency Heat Pump with Electric Heat Backup             10.7
Mininum Efficiency Whole Home Electrification                      10.7
Mininum Efficiency Heat Pump with Existing Heat Backup              9.8
Heat Pump Water Heater                                              8.8
Basic Enclosure                                                     8.7
Electric Cooking                                                    4.7
Heat Pump Clothes Dryer                            

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    46.2
Basic Enclosure + High Efficiency Whole Home Electrification       45.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     44.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        43.5
High Efficiency Whole Home Electrification                         43.0
High Efficiency Heat Pump with Electric Heat Backup                29.7
Mininum Efficiency Whole Home Electrification                      16.8
Mininum Efficiency Heat Pump with Electric Heat Backup             14.9
Mininum Efficiency Heat Pump with Existing Heat Backup             13.4
Enhanced Enclosure                                                 10.7
Heat Pump Water Heater                                              9.3
Basic Enclosure                                                     8.5
Electric Cooking                                                    2.9
Heat Pump Clothes Dryer                            

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    39.2
Basic Enclosure + High Efficiency Whole Home Electrification       38.2
High Efficiency Whole Home Electrification                         36.5
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     36.3
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        35.3
High Efficiency Heat Pump with Electric Heat Backup                19.6
Mininum Efficiency Whole Home Electrification                      12.0
Heat Pump Water Heater                                             11.7
Enhanced Enclosure                                                  6.8
Mininum Efficiency Heat Pump with Electric Heat Backup              6.1
Mininum Efficiency Heat Pump with Existing Heat Backup              6.1
Basic Enclosure                                                     5.3
Electric Cooking                                                    4.7
Heat Pump Clothes Dryer                            

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/1358844843.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/1358844843.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/1358844843.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

### Annual Emission Savings Per Dwelling Unit

In [48]:
indicator ='saving_emission.total_kgCO2e'
results = segment_mean_savings(indicator, community_results)
saving_results[f'segment1_{income}_saving_emission_mean_kgCO2e'] = results[0]
saving_results[f'segment2_{income}_saving_emission_mean_kgCO2e'] = results[1]
saving_results[f'segment3_{income}_saving_emission_mean_kgCO2e'] = results[2]
saving_results[f'segment4_{income}_saving_emission_mean_kgCO2e'] = results[3]

segment1


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    4107.7
Basic Enclosure + High Efficiency Whole Home Electrification       4047.2
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     4004.4
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        3944.0
High Efficiency Whole Home Electrification                         3634.2
High Efficiency Heat Pump with Electric Heat Backup                2985.7
Mininum Efficiency Whole Home Electrification                      2287.9
Enhanced Enclosure                                                 1865.7
Mininum Efficiency Heat Pump with Electric Heat Backup             1801.1
Basic Enclosure                                                    1600.8
Mininum Efficiency Heat Pump with Existing Heat Backup             1451.0
Heat Pump Water Heater                                              383.3
Electric Cooking                                                    110.9
Heat Pump Clothes Dryer  

segment2


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1287.3
Basic Enclosure + High Efficiency Whole Home Electrification       1262.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1223.5
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        1199.1
High Efficiency Whole Home Electrification                         1167.7
High Efficiency Heat Pump with Electric Heat Backup                 815.2
Mininum Efficiency Heat Pump with Electric Heat Backup              365.4
Mininum Efficiency Heat Pump with Existing Heat Backup              342.9
Enhanced Enclosure                                                  342.5
Mininum Efficiency Whole Home Electrification                       290.7
Basic Enclosure                                                     275.2
Heat Pump Water Heater                                              218.7
Electric Cooking                                                    109.8
Heat Pump Clothes Dryer  

segment3


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    2641.8
Basic Enclosure + High Efficiency Whole Home Electrification       2593.8
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     2528.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup        2480.0
High Efficiency Whole Home Electrification                         2441.6
High Efficiency Heat Pump with Electric Heat Backup                1736.3
Mininum Efficiency Heat Pump with Electric Heat Backup              888.4
Mininum Efficiency Heat Pump with Existing Heat Backup              830.9
Mininum Efficiency Whole Home Electrification                       825.7
Enhanced Enclosure                                                  667.4
Heat Pump Water Heater                                              480.7
Basic Enclosure                                                     479.5
Electric Cooking                                                    136.2
Heat Pump Clothes Dryer  

segment4


upgrade_name
Enhanced Enclosure + High Efficiency Whole Home Electrification    1087.4
Basic Enclosure + High Efficiency Whole Home Electrification       1047.0
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup     1019.9
High Efficiency Whole Home Electrification                          994.9
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup         979.6
High Efficiency Heat Pump with Electric Heat Backup                 575.9
Heat Pump Water Heater                                              287.5
Mininum Efficiency Whole Home Electrification                       261.3
Enhanced Enclosure                                                  227.2
Mininum Efficiency Heat Pump with Existing Heat Backup              212.7
Mininum Efficiency Heat Pump with Electric Heat Backup              209.6
Basic Enclosure                                                     157.0
Electric Cooking                                                    100.4
Electric Clothes Dryer   

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/624873452.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/624873452.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/624873452.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

## 3.5. Pecent Stock with Positive and Negtive Savings

### Segment 1

In [49]:
segment = 'segment1'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment1_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment1_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment1_{income}_pct_energy_positive_savings_%'] = results[2]*100

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/868763001.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/868763001.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/868763001.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.conca

### Segment 2

In [50]:
segment = 'segment2'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment2_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment2_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment2_{income}_pct_energy_positive_savings_%'] = results[2]*100

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/3209730133.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/3209730133.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/3209730133.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

### Segment 3

In [51]:
egment = 'segment3'
results = figure_positive_negative_savings(segment, community_results) 
saving_results[f'segment3_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment3_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment3_{income}_pct_energy_positive_savings_%'] = results[2]*100

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/1326448381.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/1326448381.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/1326448381.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

### Segment 4

In [52]:
segment = 'segment4'
results = figure_positive_negative_savings(segment, community_results)
saving_results[f'segment4_{income}_pct_bill_positive_savings_%'] = results[0]*100
saving_results[f'segment4_{income}_pct_emission_positive_savings_%'] = results[1]*100
saving_results[f'segment4_{income}_pct_energy_positive_savings_%'] = results[2]*100

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/2289506022.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/2289506022.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/2289506022.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

## 3.6. Average Energy Burden (%) Before and after Upgrade

In [53]:
results = segment_mean_energy_burden('post-upgrade_energy_burden_2023_cbills.%', community_results) 
saving_results[[f'segment1_{income}_energy_burden_before_upgrade_%',f'segment1_{income}_energy_burden_after_upgrade_%']] = results[0]
saving_results[[f'segment2_{income}_energy_burden_before_upgrade_%',f'segment2_{income}_energy_burden_after_upgrade_%']] = results[1]
saving_results[[f'segment3_{income}_energy_burden_before_upgrade_%',f'segment3_{income}_energy_burden_after_upgrade_%']] = results[2]
saving_results[[f'segment4_{income}_energy_burden_before_upgrade_%',f'segment4_{income}_energy_burden_after_upgrade_%']] = results[3]

energy burden (%) before and after upgrade
segment1


/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/3555432078.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/3555432078.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/3555432078.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,21.1,9.7
Basic Enclosure + High Efficiency Whole Home Electrification,21.1,9.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,21.1,10.0
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,21.1,10.2
High Efficiency Whole Home Electrification,21.1,11.4
High Efficiency Heat Pump with Electric Heat Backup,21.1,12.9
Enhanced Enclosure,21.1,15.7
Basic Enclosure,21.1,16.5
Mininum Efficiency Heat Pump with Electric Heat Backup,21.1,17.2


segment2


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,15.1,8.5
Basic Enclosure + High Efficiency Whole Home Electrification,15.1,8.6
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,15.1,8.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,15.1,8.8
High Efficiency Whole Home Electrification,15.1,9.1
High Efficiency Heat Pump with Electric Heat Backup,15.1,10.9
Mininum Efficiency Heat Pump with Existing Heat Backup,15.4,13.5
Mininum Efficiency Heat Pump with Electric Heat Backup,15.1,13.2
Enhanced Enclosure,15.1,13.4


segment3


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,24.7,13.8
Basic Enclosure + High Efficiency Whole Home Electrification,24.7,13.9
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,24.7,14.2
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,24.7,14.3
High Efficiency Whole Home Electrification,24.7,14.4
High Efficiency Heat Pump with Electric Heat Backup,24.7,17.2
Mininum Efficiency Heat Pump with Electric Heat Backup,24.7,20.8
Mininum Efficiency Heat Pump with Existing Heat Backup,24.7,20.9
Enhanced Enclosure,24.8,22.4


segment4


,before,after
upgrade_name,,
Enhanced Enclosure + High Efficiency Whole Home Electrification,12.5,7.6
Basic Enclosure + High Efficiency Whole Home Electrification,12.5,7.7
Enhanced Enclosure + HPWH + High Efficiency HP/Electric Backup,12.5,7.8
Basic Enclosure + HPWH + High Efficiency HP/Electric Backup,12.5,8.0
High Efficiency Whole Home Electrification,12.5,8.0
High Efficiency Heat Pump with Electric Heat Backup,12.5,9.7
Heat Pump Water Heater,12.5,11.3
Enhanced Enclosure,12.5,11.5
Mininum Efficiency Heat Pump with Existing Heat Backup,12.7,11.7


/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/2330438264.py:2: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/2330438264.py:2: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/zw/lhnn73fj2250g9z11h_whq1szkf2pm/T/ipykernel_46519/2330438264.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

In [54]:
saving_results.to_csv(fig_dir / f'saving_results_{community}.csv')